# Model operationalization via s2i

It's possible to use [s2i](https://github.com/openshift/source-to-image) to operationalize models that have been trained in a notebook.  The relevant builder image is [here](https://github.com/willb/simple-model-s2i); it works best if you follow some basic conventions.  The rest of this notebook will demonstrate these conventions with a simple example.

## requirements

The first convention to follow is declaring your model's requirements as a list of lists in a variable called `requirements`.  The s2i builder will use these to generate a `requirements.txt` file, which it will install while building an image.  This step is optional, but it is necessary if your model will depend on any libraries.

In [12]:
requirements = [["numpy", "1.15"], ["scikit-learn", "0.19.2"], ["scipy", "1.0.1"]]

## model code

Your model training code can just appear in this notebook as it would in any other.  Note that the s2i build process will execute every cell in the notebook in order.

In [11]:
import numpy as np
from sklearn.cluster import KMeans

In [10]:
DIMENSIONS = 2
randos = np.random.random((40000,DIMENSIONS))

In [21]:
kmodel = KMeans(n_clusters=7).fit(randos)

In [17]:
kmodel

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

## validate and predict

Given a trained model, you simply need to provide two functions:

* `predictor`, which will make a single prediction from a single sample, and
* `validator`, which will return `True` if a single sample is of the correct type.

In [26]:
def predictor(x):
    kmodel.predict([x])

`validator` is optional, but it will make your model service easier to use.  If you don't provide one, your model service will accept any input, which will likely lead to confusing error messages (i.e., crashes somewhere in the `predictor`) if your model service is called with bogus input.

In [27]:
def validator(x):
    return len(x) == DIMENSIONS